In [15]:
import pandas as pd 
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error
import pickle

# Q1 Downloading the data

Downloading from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, we want to 
download the yellow taxi data for Jan, Feb 2023

In [2]:
from utils import download_data_files

download_data_files(
    "yellow_tripdata_2023-01.parquet", # jan 2023
    "yellow_tripdata_2023-02.parquet", # feb 2023
)

Downloaded yellow_tripdata_2023-02.parquet with 2913955 rows and 19 columns


In [3]:
from utils import load_file 
df_jan = load_file("yellow_tripdata_2023-01.parquet")
print(f"Columns {df_jan.shape[1]}")


Columns 19


# Q2: Computing Duration

What's the standard deviation of the trips duration in January

In [4]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
def calculate_duration(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    return df 
def set_categorical_to_str(df):
    df[['PULocationID','DOLocationID']] = df[['PULocationID','DOLocationID']].astype(str)
    return df 


df_jan_cleaned = (
                    df_jan.pipe(calculate_duration)
                    .pipe(set_categorical_to_str)
)
df_jan_cleaned.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [6]:
std_dev = df_jan_cleaned.std(numeric_only=True)
print("Std Dev for duration", std_dev['duration'])


Std Dev for duration 42.59435124195458


# Q3 Dropping Outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [7]:
orig_records = df_jan_cleaned.shape[0]
print("Original Record count", orig_records)

def remove_outliers(df):
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df 

df_jan_no_outliers = df_jan.pipe(remove_outliers) 
filtered_count = df_jan_no_outliers.shape[0]
print("Shape", df_jan_no_outliers.shape)
print("Filtered count", filtered_count)
print(f"Kept {filtered_count/orig_records:2.2%}")

Original Record count 3066766
Shape (3009173, 20)
Filtered count 3009173
Kept 98.12%


# Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [8]:
# only use PULocationID and DOLocationID in model
train_dicts = df_jan_no_outliers[['PULocationID','DOLocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print("Columns", X_train.shape[1])

Columns 515


# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

 - Train a plain linear regression model with default parameters, where duration is the response variable
 - Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [9]:
target = 'duration'
y_train = df_jan_no_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261957977324

# Q6. Evaluating the Model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [10]:
df_val= load_file('yellow_tripdata_2023-02.parquet')
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [55]:
df_val_clean = (
    df_val.pipe(calculate_duration)
    .pipe(set_categorical_to_str)
    .pipe(remove_outliers)
)
print("Shape of eval data", df_val_clean.shape)
df_val_clean.head()

Shape of eval data (2855951, 20)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1.0,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148.0,236.0,1.0,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1.0,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137.0,244.0,1.0,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [57]:
val_dicts = df_val_clean[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_val = df_val_clean[target].values

y_pred_val = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred_val)

13.32268264083228

In [16]:
# save model 
os.makedirs('./models/hw01')
with open('models/hw01/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)